In [2]:
from shutil import get_terminal_size
from collections import deque
import numpy as np
from heapq import heappop, heappush
import time
import heapq
from itertools import count
import math
import random
from tabulate import tabulate
from collections import deque
import sys

**Code to visualize the problem**


In [3]:
terminal_width, _ = get_terminal_size()
_visualizers = {}

def _default_visualizer(_, state):
    print(state)

class Visualizer:

    def __init__(self, problem):
        self.problem = problem
        self.counter = 0

    def visualize(self, frontier):
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for node in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, node.state)
        print('-' * terminal_width)

    def visualize_state(self, state=None):
        if state is None:
            state = self.problem.init_state
        self.counter += 1
        print(f'Visualizing state at step {self.counter}')
        print('-' * terminal_width)


**The problem class Robot_path**

In [4]:
class Robot_Path():

      def __init__(self, init_state, goal_state, grid_size, obstacles=None):
        #assert is a function to make sure that entered initial and goal states are not outside of grid
        assert 0 <= init_state[0] < grid_size[0] and 0 <= init_state[1] < grid_size[1]
        assert 0 <= goal_state[0] < grid_size[0] and 0 <= goal_state[1] < grid_size[1]
        self._grid_size = grid_size
        self.init_state = init_state
        self._goal_state = goal_state
        self.obstacles = set(obstacles) if obstacles else set()
        self._action_values = {'up': (-1,0), 'down': (+1,0), 'left': (0,-1), 'right': (0,+1)}

      def actions(self, state):
        x,y = state
        possible_moves = []
        if x > 0:
            possible_moves.append('up')
        if x < self._grid_size[0] - 1:
            possible_moves.append('down')
        if y > 0:
            possible_moves.append('left')
        if y < self._grid_size[1] - 1:
            possible_moves.append('right')
        return possible_moves

      def result(self, state, action):
        x,y = state
        new_x,new_y = self._action_values[action]
        if (new_x,new_y) not in self.obstacles:
          return (x + new_x, y + new_y)
        else:
          return state

      def goal_test(self, state):
        return state == self._goal_state

      def step_cost(self, state, action):
        return 1

def visualize_grid(problem, state):
    #Visualizing the grid
    grid = [['.' for _ in range(problem._grid_size[1])] for _ in range(problem._grid_size[0])]
    for ox, oy in problem.obstacles:
      grid[ox][oy] = 'X'
    grid[problem._goal_state[0]][problem._goal_state[1]] = 'G'
    grid[state[0]][state[1]] = 'A'

    for row in grid:
        print(' '.join(row))

_visualizers[Robot_Path] = visualize_grid

**Generate problem**

In [5]:
n = 32
random_obs = np.random.randint(1, n, size=(n,2))
obstacles = [(x,y) for x,y in random_obs if (x,y) != (0,0) and (x,y) != (n-1,n-1)]
problem = Robot_Path((0,0),(n-1,n-1), (n,n), obstacles)
visualize_grid(problem, problem.init_state)

A . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . X
. X . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . X . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . X . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . X . . . . . . . . . . . . .
. . . X . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . X . . . . . . . . . . . . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . X . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . X . . . . . . . . . . . .
. . . . . . . X . . . . . . . . . . . . 

**Node class**

In [6]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    @classmethod
    def root(cls, state):
        return cls(state)

    @classmethod
    def child(cls, problem, parent, action):
        state = problem.result(parent.state, action)
        path_cost = parent.path_cost + problem.step_cost(parent.state, action)
        return cls(state, parent, action, path_cost)

    def __lt__(self, other):
        return self.path_cost < other.path_cost

    def expand(self, problem):
        return [Node.child(problem, self, action) for action in problem.actions(self.state)]

def solution(node):
    path = []
    while node.parent is not None:
        path.append(node.action)
        node = node.parent
    return path[::-1]

**Heuristic function manhattan and euclidean distances**

In [7]:
def heuristic_man(node):
    current = node.state
    goal = problem._goal_state
    return abs(current[0] - goal[0]) + abs(current[1] - goal[1])

def heuristic_euc(node):
    current = node.state
    goal = problem._goal_state
    return np.sqrt((current[0] - goal[0])**2 + (current[1] - goal[1])**2)

**Greedy best first graph search**

In [8]:
def greedy_best_first_graph_search(problem, h=None, verbose=False):
    h = h or problem.h
    node = Node.root(problem.init_state)
    frontier = [(h(node), node)]
    explored = set()
    max_frontier_size = 1

    visualizer = Visualizer(problem) if verbose else None

    if verbose:
        visualizer.visualize([node])

    while frontier:
        _, node = heappop(frontier)
        if problem.goal_test(node.state):
            return solution(node), max_frontier_size

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored and all(c.state != child.state for _, c in frontier):
                heappush(frontier, (h(child), child))
                if problem.goal_test(child.state):
                    return solution(child), max_frontier_size

        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize([n for _, n in frontier])

    return None, max_frontier_size


In [9]:
actions, cost = greedy_best_first_graph_search(problem, h=heuristic_man, verbose=True)
print(f"Greedy Best-First Search Solution: {actions}, Cost: {cost}")

Streaming output truncated to the last 5000 lines.
. . . . . . . . . . . . . . . . . . . . . . . . . . . X . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . X . . . . . . . . . . . . . . . X . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . X . X . . . . . . . . . . X . . . . . . . . X .
. . . . . . . . . . . . . . . . . . . . . . . . . . . X . . . .
. . . . . . . . . . . . . . . . . . . A X . . . . . . . . . . .
. . . . . . . . . . . X . . . . . . . . . . . . . . . . . . . G

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . X
. X . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . X . . . . . . . . . 

In [10]:
actions, cost = greedy_best_first_graph_search(problem, h=heuristic_euc, verbose=True)
print(f"Greedy Best-First Search Solution: {actions}, Cost: {cost}")

Streaming output truncated to the last 5000 lines.
. . . . . . . . . . . . . . . . . . . . . . . . . . . X . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . X . . . . . . . . . . . . . . . X . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . X . X . . . . . . . . . . X . . . . . . . . X .
. . . . . . . . . . . . . . . . . . . . . . . . . . . X . . . .
. . . . . . . . . . . . . . . . . . . X X . . . . . . . . . . .
. . . . . . . . . . . X . . . . . . . . . . . . . . . . . . . G

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . X
. X . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . X . . . . . . . . . 

**Function to store attributes**

In [11]:
def memoize(fn, attr_name):
    cache = {}

    def memoized_fn(node):
        if node not in cache:
            cache[node] = fn(node)
        return cache[node]

    return memoized_fn

**A* Search**

In [12]:
def astar_search(problem, h=None, verbose=False):
    h = memoize(h or problem.h, 'h')
    node = Node.root(problem.init_state)
    frontier = [(h(node), 0, node)]
    explored = set()
    max_frontier_size = 1

    visualizer = Visualizer(problem) if verbose else None

    if verbose:
        visualizer.visualize([node])

    while frontier:
        _, _, node = heappop(frontier)
        if problem.goal_test(node.state):
            return solution(node), max_frontier_size

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            f = child.path_cost + h(child)
            if child.state not in explored and all(c.state != child.state for _, _, c in frontier):
                heappush(frontier, (f, len(explored), child))
                if problem.goal_test(child.state):
                    return solution(child), max_frontier_size

        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize([n for _, _, n in frontier])

    return None, max_frontier_size


In [13]:
actions, cost = astar_search(problem, h=heuristic_man, verbose=True)
print(f"A* Search Solution: {actions}, Cost: {cost}")

Streaming output truncated to the last 5000 lines.
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . X . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . X . . . . . . . . . . . . .
. . . X . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . X . . . . . . . . . . . . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . X . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . X . . . . . . . . . . . .
. . . . . . . X . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . X . . . . . . . . . . . . . . .
. X . . . X . . . . . . . . . . . . . . . . . . . . .

In [14]:
actions, cost = astar_search(problem, h=heuristic_euc, verbose=True)
print(f"A* Search Solution: {actions}, Cost: {cost}")

Streaming output truncated to the last 5000 lines.
. . . . . . . . . . . . . . . . . . . X . . . . . . . . . . . .
. . . . . . . X . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . X . . . . . . . . . . . . . . .
. X . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . X . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . X . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . A
. . . . . . . . . . . . . X . . . . . . . . . . . . . . . X . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . X . X . . . . . . . . . . X . . . . .

**Function to measure performance of given algorithms**

In [15]:
def measure_performance(search_algorithm, problem, verbose=False):

    start_time = time.time()

    solution_result, max_frontier_size = search_algorithm(problem, verbose=verbose)

    elapsed_time = time.time() - start_time

    return {
        'solution': solution_result if solution_result else None,
        'cost': max_frontier_size if solution_result else 0,
        'elapsed_time': elapsed_time
    }

In [16]:
#run gbfs to calculate the performance
gbfs_performance_man = measure_performance(
    lambda problem, verbose: greedy_best_first_graph_search(problem, h=heuristic_man, verbose=verbose),
    problem,
    verbose=False
)

In [17]:
gbfs_performance_euc = measure_performance(
    lambda problem, verbose: greedy_best_first_graph_search(problem, h=heuristic_euc, verbose=verbose),
    problem,
    verbose=False
)

In [18]:
#run A* to calculate the performance
astar_performance_man = measure_performance(
    lambda problem, verbose: astar_search(problem, h=heuristic_man, verbose=verbose),
    problem,
    verbose=False
)

In [19]:
astar_performance_euc = measure_performance(
    lambda problem, verbose: astar_search(problem, h=heuristic_euc, verbose=verbose),
    problem,
    verbose=False
)

**Visualize the performance measures**

In [20]:
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["GBFS", gbfs_performance_man['solution'], gbfs_performance_man['cost'],
     f"{gbfs_performance_man['elapsed_time']:.4f}"],
    ["GBFS", gbfs_performance_euc['solution'], gbfs_performance_euc['cost'],
     f"{gbfs_performance_euc['elapsed_time']:.4f}"],
    ["A*", astar_performance_man['solution'], astar_performance_man['cost'],
     f"{astar_performance_man['elapsed_time']:.4f}"],
    ["GBFS", astar_performance_euc['solution'], astar_performance_euc['cost'],
     f"{astar_performance_euc['elapsed_time']:.4f}"]
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))


+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------------+
| Algorithm   | Solution                                                                                                                                                                                                                                                                                                                                                                                                                  

**Simulated Annealing**

In [21]:
def cost_fn(state, goal_state):
    return abs(state[0] - goal_state[0]) + abs(state[1] - goal_state[1])

def neighbor_fn(problem, state):
    actions = problem.actions(state)
    if actions:
        action = random.choice(actions)
        return problem.result(state, action)
    return state

def simulated_annealing(problem, initial_temp, cooling_rate, min_temp,n):
    current_state = problem.init_state
    current_cost = cost_fn(current_state, problem._goal_state)
    temperature = initial_temp

    best_state = current_state
    best_cost = current_cost

    while temperature > min_temp:
        new_state = neighbor_fn(problem, current_state)
        new_cost = cost_fn(new_state, (n-1,n-1))

        if new_cost < current_cost:
            current_state, current_cost = new_state, new_cost
        else:
            acceptance_probability = math.exp((current_cost - new_cost) / temperature)
            if random.random() < acceptance_probability:
                current_state, current_cost = new_state, new_cost

        if current_cost < best_cost:
            best_state, best_cost = current_state, current_cost

        temperature *= cooling_rate

    return best_state, best_cost

In [22]:
optimal_state, optimal_cost = simulated_annealing(
    problem=problem,
    initial_temp=1000,
    cooling_rate=0.95,
    min_temp=0.1,
    n=n
)

print("Initial state:", problem.init_state)
print("Goal state:", (n-1,n-1))
print("Optimal state reached:", optimal_state)
print("Optimal cost (Manhattan distance):", optimal_cost)

visualize_grid(problem, optimal_state)

Initial state: (0, 0)
Goal state: (31, 31)
Optimal state reached: (12, 25)
Optimal cost (Manhattan distance): 25
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . X
. X . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . X . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . X . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . X . . . . . . . . . . . . .
. . . X . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . X . . . . . . . . . . . . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . A . . . . . .
. . . . . . . . . . . . . . . X . . . . . . . . . . . .

In [23]:
#run SA to calculate the performance
sa_performance = measure_performance(
    lambda problem, verbose: simulated_annealing( problem=problem,
    initial_temp=100000,
    cooling_rate=0.95,
    min_temp=0.1,
    n=n),
    problem,
    verbose=False
)

In [24]:
#Visualize the result of the performance measure
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["SA", sa_performance['solution'], sa_performance['cost'],
     f"{sa_performance['elapsed_time']:.4f}"],
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))

+-------------+------------+--------+------------------+
| Algorithm   | Solution   |   Cost |   Time (seconds) |
+=============+============+========+==================+
| SA          | (25, 19)   |     18 |            0.001 |
+-------------+------------+--------+------------------+


**Function to reconstruct the solution path from goal**


In [25]:
def reconstruct_solution(node):
    path = []
    while node.parent is not None:
        path.append(node.action)
        node = node.parent
    path.reverse()
    return path


**Depth first**

In [26]:
def depth_first_search(problem, verbose=False):
    stack = [Node.root(problem.init_state)]
    visited = set()
    max_frontier_size = 0

    while stack:
        max_frontier_size = max(max_frontier_size, len(stack))
        node = stack.pop()

        if problem.goal_test(node.state):
            if verbose:
                print("Solution found!")
            actions = reconstruct_solution(node)
            return actions, max_frontier_size

        visited.add(node.state)

        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in visited:
                stack.append(child)

    return None, max_frontier_size


In [27]:
depth_first_search(problem, verbose=True)

Solution found!


(['right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'down',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'left',
  'down',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',
  'right',

In [28]:
#run DFS to calculate the performance
DFS_performance = measure_performance(
    lambda problem, verbose: depth_first_search(problem, verbose=verbose),
    problem,
    verbose=False
)

In [29]:
#Visualize the results of performance measures
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["DFS", DFS_performance['solution'], DFS_performance['cost'],
     f"{DFS_performance['elapsed_time']:.4f}"],
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))

+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**IDS**

In [30]:
def iterative_deepening_search(problem, verbose=False):
    max_frontier_size = 0

    for depth in range(sys.maxsize):
        stack = [(Node.root(problem.init_state), 0)]
        visited = set()

        if verbose:
            print(f"Depth: {depth}")

        while stack:
            max_frontier_size = max(max_frontier_size, len(stack))
            node, current_depth = stack.pop()

            if node.state in visited:
                continue
            visited.add(node.state)

            if problem.goal_test(node.state):
                if verbose:
                    print(f"Solution found at depth {depth}")
                actions = reconstruct_solution(node)
                return actions, max_frontier_size

            if current_depth < depth:
                for action in problem.actions(node.state):
                    child = Node.child(problem, node, action)
                    stack.append((child, current_depth + 1))

    return None, max_frontier_size


In [31]:
actions_dfs, max_frontier_dfs = iterative_deepening_search(problem, verbose=True)
print(f"DFS Solution: {actions_dfs}, Max Frontier Size: {max_frontier_dfs}")

Depth: 0
Depth: 1
Depth: 2
Depth: 3
Depth: 4
Depth: 5
Depth: 6
Depth: 7
Depth: 8
Depth: 9
Depth: 10
Depth: 11
Depth: 12
Depth: 13
Depth: 14
Depth: 15
Depth: 16
Depth: 17
Depth: 18
Depth: 19
Depth: 20
Depth: 21
Depth: 22
Depth: 23
Depth: 24
Depth: 25
Depth: 26
Depth: 27
Depth: 28
Depth: 29
Depth: 30
Depth: 31
Depth: 32
Depth: 33
Depth: 34
Depth: 35
Depth: 36
Depth: 37
Depth: 38
Depth: 39
Depth: 40
Depth: 41
Depth: 42
Depth: 43
Depth: 44
Depth: 45
Depth: 46
Depth: 47
Depth: 48
Depth: 49
Depth: 50
Depth: 51
Depth: 52
Depth: 53
Depth: 54
Depth: 55
Depth: 56
Depth: 57
Depth: 58
Depth: 59
Depth: 60
Depth: 61
Depth: 62
Depth: 63
Depth: 64
Depth: 65
Depth: 66
Depth: 67
Depth: 68
Depth: 69
Depth: 70
Depth: 71
Depth: 72
Depth: 73
Depth: 74
Depth: 75
Depth: 76
Depth: 77
Depth: 78
Depth: 79
Depth: 80
Depth: 81
Depth: 82
Depth: 83
Depth: 84
Depth: 85
Depth: 86
Depth: 87
Depth: 88
Depth: 89
Depth: 90
Depth: 91
Depth: 92
Depth: 93
Depth: 94
Depth: 95
Depth: 96
Depth: 97
Depth: 98
Depth: 99
Depth: 100

In [32]:
#run IDS to calculate the performance
IDS_performance = measure_performance(
    lambda problem, verbose: iterative_deepening_search(problem, verbose=verbose),
    problem,
    verbose=False
)

In [33]:
#Visualize the resulted performance measure
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["IDS", IDS_performance['solution'], IDS_performance['cost'],
     f"{IDS_performance['elapsed_time']:.4f}"],
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))

+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**BFS Tree**

In [34]:
def breadth_first_tree_search(problem, verbose=False):
    frontier = deque([Node.root(problem.init_state)])
    explored = set()
    max_frontier_size = 1

    if verbose:
        visualizer = Visualizer(problem)
        visualizer.visualize(frontier)

    while frontier:
        node = frontier.popleft()

        if node.state in explored:
            continue
        explored.add(node.state)

        if problem.goal_test(node.state):
            return solution(node), max_frontier_size

        frontier.extend(node.expand(problem))
        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize(frontier)

    return None, max_frontier_size


In [35]:
actions, cost = breadth_first_tree_search(problem, verbose=True)
print(f"Solution: {actions}, Cost: {cost}")

Streaming output truncated to the last 5000 lines.
. X . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . X . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . X . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . X . . . . . . . . . . . . .
. . . X . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . X . . . . . . . . . . . . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . X . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . X . . . . . . . . . . . .
. . . . . . . X . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . . . . . . . . . . . .

In [36]:
#run BFS tree to measure the performance measure
BFS_tree_performance = measure_performance(
    lambda problem, verbose: breadth_first_tree_search(problem, verbose=verbose),
    problem,
    verbose=False
)

In [37]:
#Visualize the result of performance measure
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["BFS_tree", BFS_tree_performance['solution'], BFS_tree_performance['cost'],
     f"{BFS_tree_performance['elapsed_time']:.4f}"],
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))

+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------------+
| Algorithm   | Solution                                                                                                                                                                                                                                                                                                                                                                                                                  

**BFS graph**

In [38]:
def breadth_first_graph_search(problem, verbose=False):
    node = Node.root(problem.init_state)

    if problem.goal_test(node.state):
        return solution(node)

    frontier = deque([node])
    explored = set()
    max_frontier_size = 1

    if verbose:
        visualizer = Visualizer(problem)
        visualizer.visualize(frontier)

    while frontier:
        node = frontier.popleft()

        if node.state in explored:
            continue
        explored.add(node.state)

        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)

            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    return solution(child), max_frontier_size
                frontier.append(child)

        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize(frontier)

    return None, max_frontier_size


In [39]:
actions, cost= breadth_first_graph_search(problem, verbose=True)
print(f"Solution: {actions}, Cost: {cost}")

Streaming output truncated to the last 5000 lines.
. . . . . . . . . . . X . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . X . . . . . . . . . . . . . . .
. X . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . X . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . X . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . X . . . . . . . . . . . . . . . X . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . . . . . . . . . . . . . . A .
. . . . . . . . X . X . . . . . . . . . . X . . . . . . . . X .
. . . . . . . . . . . . . . . . . . . . . . . . . . . X . . . .
. . . . . . . . . . . . . . . . . . . X X . . . . . .

In [40]:
#run BFS graph to measure performance
BFS_graph_performance = measure_performance(
    lambda problem, verbose: breadth_first_graph_search(problem, verbose=verbose),
    problem,
    verbose=False
)

In [41]:
#Visualize the result of performance measure
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["BFS_graph", BFS_graph_performance['solution'], BFS_graph_performance['cost'],
     f"{BFS_graph_performance['elapsed_time']:.4f}"],
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))

+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------------+
| Algorithm   | Solution                                                                                                                                                                                                                                                                                                                                                                                                                  

**UCS**

In [42]:
def uniform_cost_search(problem, verbose=False):
    counter = count()
    frontier = []
    heapq.heappush(frontier, (0, next(counter), Node.root(problem.init_state)))
    explored = set()
    max_frontier_size = 1

    if verbose:
        visualizer = Visualizer(problem)
        visualizer.visualize([node for _, _, node in frontier])

    while frontier:
        cost, _, node = heapq.heappop(frontier)

        if problem.goal_test(node.state):
            return solution(node), cost

        if node.state not in explored:
            explored.add(node.state)

            for action in problem.actions(node.state):
                child = Node.child(problem, node, action)
                child_cost = cost + problem.step_cost(node.state, action)

                if child.state not in explored:
                    heapq.heappush(frontier, (child_cost, next(counter), child))

        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize([node for _, _, node in frontier])

    return None, max_frontier_size


In [43]:
actions, cost = uniform_cost_search(problem, verbose=True)
print(f"Solution: {actions}, Cost: {cost}")

Streaming output truncated to the last 5000 lines.
. . . . . . . . . . . . . . . . . . . X X . . . . . . . . . . .
. . . . . . . . . . . X . . . . . . . . . . . . . . . . A . . G

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . X
. X . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . X . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . X . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . X . . . . . . . . . . . . .
. . . X . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . X . . . . . . . . . . . . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . 

In [44]:
#run UCS to measure performance
UCS_performance = measure_performance(
    lambda problem, verbose: uniform_cost_search(problem, verbose=verbose),
    problem,
    verbose=False
)

In [45]:
#Visualize the result of performnace measure
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["UCS", UCS_performance['solution'], UCS_performance['cost'],
     f"{UCS_performance['elapsed_time']:.4f}"],
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))

+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------------+
| Algorithm   | Solution                                                                                                                                                                                                                                                                                                                                                                                                                  

**Hill Climbing**

In [46]:
def hill_climbing(problem, verbose=False):
    current = Node.root(problem.init_state)
    viz = Visualizer(problem)

    if verbose:
      visualize_grid(problem, current.state)

    visited = set()
    visited.add(current.state)

    max_iterations = 1000
    iteration = 0
    cost = 0

    while iteration < max_iterations:
        neighbors = [Node.child(problem, current, action) for action in problem.actions(current.state)]

        if not neighbors:
            break

        neighbors = [neighbor for neighbor in neighbors if neighbor.state not in visited]

        if not neighbors:
            break

        def manhattan_heuristic(state, goal_state):
            x1, y1 = state
            x2, y2 = goal_state
            return abs(x1 - x2) + abs(y1 - y2)

        next_node = min(neighbors, key=lambda node: manhattan_heuristic(node.state, problem._goal_state))

        if manhattan_heuristic(next_node.state, problem._goal_state) >= manhattan_heuristic(current.state, problem._goal_state):
            break

        current = next_node
        visited.add(current.state)

        if verbose:
          print(f"Iteration {iteration + 1}:")
          visualize_grid(problem, current.state)
          print("\n")

        iteration += 1
        cost += 1

    return current, cost


In [47]:
final_node = hill_climbing(problem, True)
print(f"Final Node: {final_node[0].state}")

A . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . X
. X . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . X . . . . X . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . X . . . . . . . . . . . . X . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . X . . . . . . . . . . . . .
. . . X . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . X . . . . . . . . . . . . . . .
. . . . . X . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . X . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . X . . . . . . . . . . . .
. . . . . . . X . . . . . . . . . . . . 

**Performance measure for hill climbing**

In [48]:
def measure_performance2(search_algorithm, problem):
    start_time = time.time()

    solution_result, cost = search_algorithm(problem)

    elapsed_time = time.time() - start_time

    return {
        'solution': solution_result if solution_result else None,
        'cost': cost if solution_result else 0,
        'elapsed_time': elapsed_time
    }

In [51]:
#run hill climbing to measure performance
hill_climbing_performance = measure_performance2(
    lambda problem: hill_climbing(problem, verbose=False),
    problem
)

In [52]:
#Visualize the result of performance measure
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["Hill Climbing", hill_climbing_performance['solution'].state, hill_climbing_performance['cost'],
     f"{hill_climbing_performance['elapsed_time']:.4f}"],
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))

+---------------+------------+--------+------------------+
| Algorithm     | Solution   |   Cost |   Time (seconds) |
+===============+============+========+==================+
| Hill Climbing | (31, 31)   |     62 |           0.0009 |
+---------------+------------+--------+------------------+


**Genetic Search**

In [53]:
class GeneticSearch:

    def __init__(self, problem, population_size=100, generations=1000, mutation_rate=0.1):
        self.problem = problem
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.visualizer = Visualizer(problem)

    def fitness(self, path):
        state = self.problem.init_state
        cost = 0

        for action in path:
          state_ = self.problem.result(state, action)
          if state_ == state:
            cost +=1
          state = state_

        dist = abs(state[0] - self.problem._goal_state[0]) + abs(state[1] - self.problem._goal_state[1])
        fitness = -(dist + cost)
        return fitness

    def create_population(self):
        actions = list(self.problem._action_values.keys())
        max_path_length = self.problem._grid_size[0] * self.problem._grid_size[1]
        return [[random.choice(actions) for _ in range(max_path_length)] for _ in range(self.population_size)]


    def select_parents(self, population, fitness_scores):
        total_fitness = sum(fitness_scores)
        probabilities = [score / total_fitness for score in fitness_scores]
        return random.choices(population, probabilities, k=2)

    def crossover(self, parent1, parent2):
        n = len(parent1)
        point = random.randint(1, len(parent1) - 1)
        return parent1[:point] + parent2[point:]

    def mutate(self, state):
        if random.random() < self.mutation_rate:
            index = random.randint(0, len(state) - 1)
            actions = self.problem._action_values.keys()
            state[index] = random.choice(list(actions))
        return state

    def run(self):
        population = self.create_population()
        for generation in range(self.generations):
            fitness_scores = [self.fitness(ind) for ind in population]

            best_fitness = max(fitness_scores)
            best_path = population[fitness_scores.index(best_fitness)]

            final_state = self.problem.init_state
            for action in best_path:
                final_state = self.problem.result(final_state, action)

            if self.problem.goal_test(final_state):
                print(f'Solution found at generation {generation + 1}')
                visualize_grid(problem, final_state)
                return best_path

            if generation % 10 == 0:
                print(f'Generation {generation}, Best Fitness: {best_fitness}')
                self.visualizer.visualize_state(final_state)

            new_population = []
            for _ in range(self.population_size):
                parent1, parent2 = self.select_parents(population, fitness_scores)
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                new_population.append(child)
            population = new_population

        print('No solution found.')
        return None


In [54]:
genetic_solver = GeneticSearch(problem=problem)
solution = genetic_solver.run()

Generation 0, Best Fitness: -4
Visualizing state at step 1
----------------------------------------------------------------------------------------------------
Generation 10, Best Fitness: -68
Visualizing state at step 2
----------------------------------------------------------------------------------------------------
Generation 20, Best Fitness: -114
Visualizing state at step 3
----------------------------------------------------------------------------------------------------
Generation 30, Best Fitness: -156
Visualizing state at step 4
----------------------------------------------------------------------------------------------------
Generation 40, Best Fitness: -190
Visualizing state at step 5
----------------------------------------------------------------------------------------------------
Generation 50, Best Fitness: -202
Visualizing state at step 6
----------------------------------------------------------------------------------------------------
Generation 60, Best Fitnes

**Function to measure the performance of Genetic algorithm**

In [55]:
def measure_performance2(search_algorithm, problem):
    start_time = time.time()

    solution_result = search_algorithm(problem)

    elapsed_time = time.time() - start_time

    return {
        'solution': solution_result if solution_result else None,
        'cost': cost if solution_result else 0,
        'elapsed_time': elapsed_time
    }

In [56]:
#run Genetic to calculate the performance measure
genetic_performance = measure_performance2(
    lambda problem: solution,
    problem
)

In [57]:
#Visualize the result of performance measure
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["Genetic", genetic_performance['solution'], genetic_performance['cost'],
     f"{genetic_performance['elapsed_time']:.4f}"]
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))

+-------------+------------+--------+------------------+
| Algorithm   | Solution   |   Cost |   Time (seconds) |
+=============+============+========+==================+
| Genetic     |            |      0 |                0 |
+-------------+------------+--------+------------------+


**Visualize the perfomance measure of all algorithms**

In [58]:
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)"]
table_data = [
    ["GBFS", gbfs_performance_man['solution'], gbfs_performance_man['cost'],
     f"{gbfs_performance_man['elapsed_time']:.4f}"],
    ["GBFS", gbfs_performance_euc['solution'], gbfs_performance_euc['cost'],
     f"{gbfs_performance_euc['elapsed_time']:.4f}"],
    ["A*", astar_performance_man['solution'], astar_performance_man['cost'],
     f"{astar_performance_man['elapsed_time']:.4f}"],
    ["GBFS", astar_performance_euc['solution'], astar_performance_euc['cost'],
     f"{astar_performance_euc['elapsed_time']:.4f}"],
    ["SA", sa_performance['solution'], sa_performance['cost'],
     f"{sa_performance['elapsed_time']:.4f}"],
    ["BFS_graph", BFS_graph_performance['solution'], BFS_graph_performance['cost'],
     f"{BFS_graph_performance['elapsed_time']:.4f}"],
    ["BFS_tree", BFS_tree_performance['solution'], BFS_tree_performance['cost'],
     f"{BFS_tree_performance['elapsed_time']:.4f}"],
    ["DFS", DFS_performance['solution'], DFS_performance['cost'],
     f"{DFS_performance['elapsed_time']:.4f}"],
    ["IDS", IDS_performance['solution'], IDS_performance['cost'],
     f"{IDS_performance['elapsed_time']:.4f}"],
    ["UCS", UCS_performance['solution'], UCS_performance['cost'],
     f"{UCS_performance['elapsed_time']:.4f}"],
    ["Hill Climbing", hill_climbing_performance['solution'].state, hill_climbing_performance['cost'],
     f"{hill_climbing_performance['elapsed_time']:.4f}"],
    ["Genetic", genetic_performance['solution'], genetic_performance['cost'],
     f"{genetic_performance['elapsed_time']:.4f}"]
]

print(tabulate(table_data, headers=headers, tablefmt='grid'))

+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------